In [31]:
import pandas as pd
import seaborn as sb
import numpy 

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor


In [2]:
#reading in csv files that we will use
multi_df = pd.read_csv('Multi_Value_Numbeo.csv') 
quality_df = pd.read_csv('Quality_of_Life_Numbeo.csv')
property_df = pd.read_csv('Property_Prices_Numbeo.csv')

In [3]:
#Checking to see if there are any null - empty values in dataset
print(multi_df.isnull().sum())
print("-----------------------------------")
print(quality_df.isnull().sum())
print("-----------------------------------")
print(property_df.isnull().sum())

Rank                            0
City                            0
Cost of Living Index            0
Rent Index                      0
Cost of Living Plus Rent        0
Groceries Index                 0
Resaurant Price Index           0
Local Purchasing Power Index    0
dtype: int64
-----------------------------------
Rank                              0
City                              0
Quality of Life Index             0
Purchasing Power Index            0
Safety Index                      0
Health Care Index                 0
Cost of Living Index              0
Property Price to Income Ratio    0
Traffic Commute Time Index        0
Pollution Index                   0
Climate Index                     0
dtype: int64
-----------------------------------
Rank                                          0
City                                          0
Price To Income Ratio                         0
Gross Rental Yield City Centre                0
Gross Rental Yield Outside of Centre      

In [4]:
#checking to see how many rows are in the df
print("Multi_df",multi_df.shape[0])
print("Quality_df",quality_df.shape[0])
print("Property_df",property_df.shape[0])

Multi_df 573
Quality_df 250
Property_df 480


if 'Adelaide, Australia' not in multi_df.City:
    print("yes")

In [5]:
multi_df = multi_df.drop("Rank",axis=1)
quality_df = quality_df.drop("Rank",axis=1)
property_df = property_df.drop("Rank",axis=1)

In [6]:
df = pd.merge(multi_df, property_df, on='City')

In [7]:
df.shape[0]

440

In [8]:
df_withLife = pd.merge(df, quality_df, how='inner',on='City')

In [9]:
df_withLife.shape[0]

241

In [10]:
df_withLife.head()

,City,Cost of Living Index_x,Rent Index,Cost of Living Plus Rent,Groceries Index,Resaurant Price Index,Local Purchasing Power Index,Price To Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside of Centre,...,Affordability Index,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index_y,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index
0,"Zurich, Switzerland",132.00,65.85,100.30,133.03,120.68,123.01,8.10,3.29,3.80,...,2.18,199.53,123.01,83.87,75.94,132.00,8.10,33.53,16.63,81.48
1,"Geneva, Switzerland",128.14,66.41,98.56,127.77,119.58,114.18,9.25,3.23,3.55,...,1.83,186.60,114.18,72.59,73.08,128.14,9.25,25.49,25.99,82.61
2,"New York, NY, United States",100.00,100.00,100.00,100.00,100.00,100.00,10.39,4.85,5.43,...,1.34,136.75,100.00,53.84,61.59,100.00,10.39,43.13,59.11,79.66
3,"Oslo, Norway",95.41,40.20,68.95,90.14,94.37,87.20,10.92,3.13,3.59,...,1.39,164.11,87.20,64.16,75.26,95.41,10.92,31.92,24.79,59.99
4,"Honolulu, HI, United States",95.08,63.69,80.04,101.17,79.75,75.98,9.35,5.42,6.53,...,1.46,154.85,75.98,55.43,72.42,95.08,9.35,41.19,36.82,95.33


In [11]:
df_withLife.columns

Index(['City', 'Cost of Living Index_x', 'Rent Index',
       'Cost of Living Plus Rent', 'Groceries Index', 'Resaurant Price Index',
       'Local Purchasing Power Index', 'Price To Income Ratio',
       'Gross Rental Yield City Centre',
       'Gross Rental Yield Outside of Centre',
       'Price To Rent Ratio City Centre',
       'Price To Rent Ratio Outside Of City Centre',
       'Mortgage As A Percentage Of Income', 'Affordability Index',
       'Quality of Life Index', 'Purchasing Power Index', 'Safety Index',
       'Health Care Index', 'Cost of Living Index_y',
       'Property Price to Income Ratio', 'Traffic Commute Time Index',
       'Pollution Index', 'Climate Index'],
      dtype='object')

In [12]:
df_withLife.dtypes

City                                           object
Cost of Living Index_x                        float64
Rent Index                                    float64
Cost of Living Plus Rent                      float64
Groceries Index                               float64
Resaurant Price Index                         float64
Local Purchasing Power Index                  float64
Price To Income Ratio                         float64
Gross Rental Yield City Centre                float64
Gross Rental Yield Outside of Centre          float64
Price To Rent Ratio City Centre               float64
Price To Rent Ratio Outside Of City Centre    float64
Mortgage As A Percentage Of Income            float64
Affordability Index                           float64
Quality of Life Index                         float64
Purchasing Power Index                        float64
Safety Index                                  float64
Health Care Index                             float64
Cost of Living Index_y      

In [13]:
#encode city column values and give them actual int values 
df_withLife["City"] = df_withLife["City"].astype('category')
df_withLife["City_Codes"] = df_withLife["City"].cat.codes



In [14]:
df_withLife.dtypes

City                                          category
Cost of Living Index_x                         float64
Rent Index                                     float64
Cost of Living Plus Rent                       float64
Groceries Index                                float64
Resaurant Price Index                          float64
Local Purchasing Power Index                   float64
Price To Income Ratio                          float64
Gross Rental Yield City Centre                 float64
Gross Rental Yield Outside of Centre           float64
Price To Rent Ratio City Centre                float64
Price To Rent Ratio Outside Of City Centre     float64
Mortgage As A Percentage Of Income             float64
Affordability Index                            float64
Quality of Life Index                          float64
Purchasing Power Index                         float64
Safety Index                                   float64
Health Care Index                              float64
Cost of Li

In [15]:
#basic first model

#selecting the prediction target 
y = df_withLife.City_Codes

#choosing features to use (selecting a few as of right now)
features = ['Quality of Life Index','Purchasing Power Index','Safety Index','Health Care Index','Cost of Living Index_y','Pollution Index']

In [16]:
X = df_withLife[features]

In [17]:
X.describe()

,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index_y,Pollution Index
count,241.000000,241.000000,241.000000,241.000000,241.000000,241.000000
mean,139.976929,70.935062,56.564938,66.608174,57.517801,51.101328
std,35.544522,32.767029,14.991353,10.020906,21.365555,21.259635
min,45.520000,10.750000,17.860000,40.520000,20.460000,12.870000
25%,111.850000,42.070000,46.660000,59.630000,37.730000,33.120000
50%,143.750000,66.890000,56.750000,67.340000,63.190000,50.220000
75%,169.810000,96.730000,68.130000,74.390000,74.410000,67.450000
max,206.410000,158.980000,88.560000,87.170000,132.000000,95.920000


In [18]:
#instantiating model specified with random_state = 1 to ensure same results each run
df_model = DecisionTreeRegressor(random_state=1)


In [19]:
#Fit model
df_model.fit(X,y)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
                      max_leaf_nodes=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      presort=False, random_state=1, splitter='best')

In [20]:
print(X.head())

   Quality of Life Index  Purchasing Power Index  Safety Index  \
0                 199.53                  123.01         83.87   
1                 186.60                  114.18         72.59   
2                 136.75                  100.00         53.84   
3                 164.11                   87.20         64.16   
4                 154.85                   75.98         55.43   

   Health Care Index  Cost of Living Index_y  Pollution Index  
0              75.94                  132.00            16.63  
1              73.08                  128.14            25.99  
2              61.59                  100.00            59.11  
3              75.26                   95.41            24.79  
4              72.42                   95.08            36.82  


In [21]:
print("Predictions are", df_model.predict(X.head()))

Predictions are [240.  76. 149. 159.  89.]


In [22]:
df_withLife.loc[df_withLife['City_Codes'] == 240]

,City,Cost of Living Index_x,Rent Index,Cost of Living Plus Rent,Groceries Index,Resaurant Price Index,Local Purchasing Power Index,Price To Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside of Centre,...,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index_y,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,City_Codes
0,"Zurich, Switzerland",132.0,65.85,100.3,133.03,120.68,123.01,8.1,3.29,3.8,...,199.53,123.01,83.87,75.94,132.0,8.1,33.53,16.63,81.48,240


In [23]:
#validating prediction print MAE 
predicted_city = df_model.predict(X)
mean_absolute_error(y, predicted_city)

0.0

In [24]:
train_X, val_X, train_y, val_y = train_test_split(X,y,random_state=0)
#define model
df_model = DecisionTreeRegressor()
#fit model
df_model.fit(train_X,train_y)

#retrieve predicted city on validated data
val_predictions = df_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

85.59016393442623


In [25]:
#predicted city is about 90 off 

In [26]:
forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(train_X, train_y)
df_prediction = forest_model.predict(val_X)
print(mean_absolute_error(val_y, df_prediction))

67.78032786885245


/opt/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [33]:
numpy.round(df_prediction)

array([130.,  70., 104.,  93., 149., 126.,  98., 125.,  86.,  60.,  66.,
        96.,  64., 117.,  90., 122., 142., 129., 125., 111.,  97., 109.,
       142., 157., 153.,  81., 118.,  85.,  72., 153., 124.,  67.,  75.,
       160., 102., 144., 165., 136., 131., 166., 100., 143., 126.,  96.,
       112., 155., 102., 192., 125.,  87., 110., 119., 150., 112., 137.,
       102., 112.,  90., 134., 137., 120.])

In [39]:
df_withLife.loc[df_withLife['City_Codes'] == 126]

,City,Cost of Living Index_x,Rent Index,Cost of Living Plus Rent,Groceries Index,Resaurant Price Index,Local Purchasing Power Index,Price To Income Ratio,Gross Rental Yield City Centre,Gross Rental Yield Outside of Centre,...,Quality of Life Index,Purchasing Power Index,Safety Index,Health Care Index,Cost of Living Index_y,Property Price to Income Ratio,Traffic Commute Time Index,Pollution Index,Climate Index,City_Codes
35,"Lyon, France",78.93,26.9,54.0,74.98,67.47,80.18,9.3,3.19,3.78,...,153.81,80.18,55.28,77.39,78.93,9.3,33.78,48.4,88.51,126
